In [2]:
%pip install langchain-core langchain-community

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [1]:
MODEL = 'llama2'


In [4]:

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings=OllamaEmbeddings()


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [31]:
!ls

01-run-ollama.ipynb  02-langchain.ipynb


In [27]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("../pdfs/mlschool.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='Building Machine Learning Systems That Don\'t Suck\nA live, interactive program that\'ll help you build production-readymachine learning\nsystems from the ground up.\nNext cohort:\xa0July1 - 18, 2024\nCheck the schedulefor more details about upcoming cohorts.\nI want to join!Sign in\nLearn how to design, build, deploy, and scale machine learning\nsystems to solve real-world problems.\nI\'ll lose my mind if I see another book or course teaching people the same basic ideas\nfor the hundredth time. Most people are stuck in beginner mode, and finding help to\nsolve real-world problems is hard.\nI want to change that.\nI started writing software 30 years ago. I\'ve written pipelines and trained models for\nsome of the largest companies in the world. I want to show you how to do the same.\nThis is the class I wish I had taken when I started."This is the best machine learning course I\'ve done.\nWorth every cent."\n— Jose Reyes, AI/ML at Cevo Australia5/30/24, 12:52 PM

In [6]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
# print(prompt.format(context="Here is some context", question="This is the question"))



In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()



{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [28]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

In [34]:
retriever = vectorstore.as_retriever()
retriever.invoke("ML")

[Document(page_content="Active learning using the uncertainty and diversity sampling strategies.\nSession 2 - How to Build a Model\nThe role of data cleaning and feature engineering to build better models.\nTurning data into numbers using vectorization techniques.\nProducing homogeneous features using normalization and standardization.\nHandling and interpreting missing values using imputation techniques.\nThe approach to choosing the best model to solve any problem.\nRandom baselines and the zero-rule algorithm.\nHow to use overfitting to build models that don't suck.\nHyperparameter tuning and experiment tracking.\nMeasuring the quality of your holdout set.\nAn introduction to distributed training using data parallelism and model\nparallelism.\nSession 3 - Offline Evaluation\nThe role of a baseline to contextualize the evaluation process.\nFraming evaluation metrics in the context of business performance.\nEvaluating models using holdout sets and cross-validation.\nIntroduction to da

In [35]:
output = chain.invoke({"context":"The name I was given was Deepak", "question":"What's my name"})
print(output)
# for token in output:
#     print(token, end="", flush=True)

I don't know your name as I'm just an AI and do not have access to personal information or the ability to know the identity of individuals. My purpose is to assist with tasks and answer questions based on the content provided to me, but I do not have personal knowledge or context beyond what is presented in the text. Is there anything else I can help you with?


In [36]:

from operator import itemgetter



chain = (
    {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question")
    } 
    | prompt 
    | model 
    | parser)
chain.input_schema.schema()


{'title': 'RunnableParallel<context,question>Input',
 'type': 'object',
 'properties': {'question': {'title': 'Question'}}}

In [37]:
# questions = [
#     "What is this document about?",
#     "What GPUs are being compared?",
# ]
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
    "Is there a program certificate upon completion?",
    "What programming language will be used in the program?",
    "How much does the program cost?",
]


In [38]:
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: ", end="")
    for token in chain.stream({"question" : question}):
        print(token, end="", flush=True)
        
    print()
    

Question: What is the purpose of the course?
Answer: Based on the provided context, the purpose of the course appears to be teaching students practical skills and insights to build machine learning systems that work in real-world scenarios. The course aims to provide students with hands-on experience through live, interactive sessions, step-by-step coding instructions, and access to complete source code of a production system. Additionally, the course offers support from the community and direct feedback from the instructor to help students learn and grow.
Question: How many hours of live sessions?
Answer: Based on the context provided, there are 18 hours of live sessions in the program.
Question: How many coding assignments are there in the program?
Answer: Based on the context provided, there are 100 coding assignments in the program.
Question: Is there a program certificate upon completion?
Answer: Yes, upon completion of the program, you will receive a program certificate.
Question